# Funciones extras
En este Jupyter Notebook vamos a describir algunas funciones complementarias que usamos para los procesos de limpieza

## Función para arreglar las fechas
En esta función se transforman todas las fechas de cualquier otro formato a "DD/MM/YYYY"

In [ ]:
# Leemos toda la columna de fechas y almacenamos todas las fechas en una lista
fechas = df['FECHA_INTERVENCION'].tolist()

# Ahora, pasamos por todas las fechas y corregimos las fechas que están mal escritas
fechas_corregidas = []
año_al_final = False

for fecha in fechas:
    # Si fecha contiene un guion, hacer fecha.split('-') y si fecha contiene una barra, hacer fecha.split('/')
    if '-' in fecha:
        fecha_split = fecha.split('-')
    elif '/' in fecha:
        fecha_split = fecha.split('/')
    # Condiciones para saber el día, mes y año
    # Encontrar el año
    if (int(fecha_split[0])>31):
        año = fecha_split[0]
        año_al_final = False
    elif (int(fecha_split[2])>31):
        año = fecha_split[2]
        año_al_final = True
    else:
        print("No se ha encontrado el año")
        fechas_corregidas.append("fecha incorrecta")
        break
    # Encontrar el día y el mes
    # Si el año está al final, no comprobamos el primer caracter
    if año_al_final:
        if (int(fecha_split[0])<32 and int(fecha_split[0])>12):
            dia = fecha_split[0]
            mes = fecha_split[1]
        elif (int(fecha_split[1])<32 and int(fecha_split[1])>12):
            dia = fecha_split[1]
            mes = fecha_split[0]
        # Si no hay ningun número entre el 13 y el 31, se asume que el mes es el segundo siempre y el día es el primero
        else:
            dia = fecha_split[0]
            mes = fecha_split[1]
    else:
        if (int(fecha_split[1])<32 and int(fecha_split[1])>12):
            dia = fecha_split[1]
            mes = fecha_split[2]
        elif (int(fecha_split[2])<32 and int(fecha_split[2])>12):
            dia = fecha_split[2]
            mes = fecha_split[1]
        # Si no hay ningun número entre el 13 y el 31, se asume que el mes es el segundo siempre y el día es el tercero
        else:
            dia = fecha_split[2]
            mes = fecha_split[1]
    fechas_corregidas.append(dia+"/"+mes+"/"+año)
    
# Ahora, cambiamos toda la columna de FECHAS_INTERVENCION por las fechas corregidas
df['FECHA_INTERVENCION'] = fechas_corregidas

# Mostramos las fechas corregidas
print("   ----------    Columna de fechas corregidas    ----------    ")
print("\n")
print(df['FECHA_INTERVENCION'].sample(20))

## Función para transformar los valores nulos según su ID y el nombre de la columna

In [ ]:
# Elimina todos los guiones y espacios de todos los valores de la columna 'ID'
# Función para generar la cadena personalizada
def custom_string(row):
    main_string = row['ID'][1:-7]+row['ID'][-3:] 
    return main_string

# Reemplazar valores nulos
df['ID'] = df.apply(lambda row: custom_string(row), axis=1)

print(df['ID'].head(40))

In [ ]:
# Cambiamos los valores nulos por una string personalizada en la que combinamos el NIF de esa fila y el nombre de la columna en cuestión

# Función para generar la cadena personalizada
def string_email_desc(row):
    return f"{row['NIF']}_EMAIL_Desconocido"

# Reemplazar valores nulos
df['EMAIL'] = df.apply(lambda row: string_email_desc(row) if pd.isnull(row['EMAIL']) else row['EMAIL'], axis=1)

# Mostrar el DataFrame con los valores nulos reemplazados
print(df.sample(20))

print("   ----------    Filas con valores nulos    ----------    ")
print(df[df.isnull().any(axis=1)])

# Función para quitar las tildes en general

In [ ]:
# Limpiamos la columna de 'TIPO_INCIDENTE' para que no haya tildes en lugares raros
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Cáídá', 'Caída')
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Róbó', 'Robo')
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Accídénté', 'Accidente')
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Vándálísmó', 'Vandalismo')
df['TIPO_INCIDENTE'] = df['TIPO_INCIDENTE'].str.replace('Dáñó éstrúctúrál', 'Daño estructural')

# Muestra todos los valores distintos de la columna 'TIPO_INCIDENTE'
print("   ----------    Valores distintos de la columna 'TIPO_INCIDENTE'    ----------    ")
print("\n")
print(df['TIPO_INCIDENTE'].unique())

# Función para arreglar las filas repetidas

In [ ]:
# Filtrar filas que tienen el mismo NIF
duplicados = df.groupby('ID').filter(lambda x: len(x) > 1)

# Ordenar por NIF para que las filas con el mismo NIF se visualicen una encima de la otra
duplicados = duplicados.sort_values(by='ID')

# Mostrar las filas con la misma PK
print("   ----------    Filas con el mismo ID    ----------    ")
print("\n")
print(duplicados)
print("\n")
print("Número de filas con el mismo ID: ", duplicados.shape[0])
print("\n")

# Limpiamos una de las dos filas con el mismo ID de duplicados
df.drop_duplicates(subset='ID', keep='first', inplace=True)

# Mostramos el resultado por pantalla
print("   ----------    Filas con el mismo ID después de limpiar    ----------    ")
print("\n")
print(df[df.duplicated()])
print("\n")
# Número de filas duplicadas
print("Número de filas duplicadas: ", df.duplicated().sum())
print("\n")

# Mostramos, para comprobar, 2 filas de las que antes estaban duplicadas
print("   ----------    2 filas de las que antes estaban duplicadas    ----------    ")
print("\n")
print(df.loc[df['ID'] == 431])
print(df.loc[df['ID'] == 9948])
print("\n")

# Mostramos el intento de acceder a una fila eliminada
print("   ----------    Intento de acceder a una fila eliminada    ----------    ")
print("\n")
# Accedemos a la fila con ID 431 y FECHA_REPORTE 14/08/2024
print(df.loc[(df['ID'] == 431) & (df['FECHA_REPORTE'] == '14/08/2024')])